<a href="https://colab.research.google.com/github/danabuogi/Machine-Learning-Classification/blob/main/ML_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [ ]:
csv_column_names = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Species"]
species = ["Setosa", "Versicolor", " Virginica"]

In [ ]:
train_path = tf.keras.utils.get_file("iris_training_csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test_csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
train = pd.read_csv(train_path, names = csv_column_names, header = 0)
test = pd.read_csv(test_path, names = csv_column_names, header = 0)

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
y_train = train.pop("Species")
y_test = test.pop("Species")
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
def input_fn(features, labels, training = True, batch_size = 256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training:
    dataset= dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [ ]:
feature_columns = []
for key in train.keys(): # train.keys gives the columns
  feature_columns.append(tf.feature_column.numeric_column(key=key))
print(feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
# build DNN with 2 hidden layers with 30 and 10 hidden nodes each and it has to choose between 3 classes
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, hidden_units = [30,10], n_classes=3)

In [ ]:
classifier.train(input_fn=lambda: input_fn(train, y_train, training = True), steps=5000)

In [ ]:
evaluation_result = classifier.evaluate(input_fn=lambda: input_fn(test, y_test, training=False))
print("\nTest set accuracy: {accuracy:0.3f}\n".format(**evaluation_result))


Test set accuracy: 0.600



In [ ]:
def my_input_fn(features, batch_size = 256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: my_input_fn(predict))
for pre_dict in predictions:
  class_id = pre_dict["class_ids"][0]
  probability = pre_dict["probabilities"][class_id]

  print("Prediction is '{}' ({: .1f}%)".format(species[class_id], 100 * probability))

Please type numeric values as prompted.
SepalLength: 2.0
SepalWidth: 2.3
PetalLength: 6.1
PetalWidth: 6.3
Prediction is ' Virginica' ( 57.7%)
